# CPT Quantile DDPG with PettingZoo Integration

This notebook demonstrates how to integrate the CPT-Quantile-DDPG agent in a competitive multi-agent environment using PettingZoo. For each agent in the environment, we create an instance of the agent and its replay buffer, then run a training loop using PettingZoo’s AEC API.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from pettingzoo.mpe import simple_tag_v2

## --- Agent Code (same as in the first notebook) ---
def cpt_transform_tensor(rewards, alpha=0.88, beta=0.88, lambda_=2.25):
    rewards = rewards.float()
    pos = torch.pow(torch.clamp(rewards, min=0), alpha)
    neg = -lambda_ * torch.pow(-torch.clamp(rewards, max=0), beta)
    return pos + neg

class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
            nn.Tanh()
        )
    def forward(self, state):
        return self.net(state)

class QuantileCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=256, num_quantiles=50):
        super().__init__()
        self.num_quantiles = num_quantiles
        self.net = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.quantile_layer = nn.Linear(hidden_dim, num_quantiles)
    def forward(self, state, action):
        x = torch.cat([state, action], dim=-1)
        x = self.net(x)
        quantiles = self.quantile_layer(x)
        return quantiles

def huber(x, kappa=1.0):
    abs_x = x.abs()
    cond = (abs_x <= kappa).float()
    loss = 0.5 * (x ** 2) * cond + kappa * (abs_x - 0.5 * kappa) * (1 - cond)
    return loss

def quantile_regression_loss(predicted, target, taus, kappa=1.0):
    batch_size, num_quantiles = predicted.shape
    target_exp = target.unsqueeze(2)
    predicted_exp = predicted.unsqueeze(1)
    error = target_exp - predicted_exp
    taus = taus.view(1, num_quantiles, 1)
    indicator = (error < 0).float()
    loss = torch.abs(taus - indicator) * huber(error, kappa)
    return loss.mean()

class CPTQuantileDDPG:
    def __init__(self, state_dim, action_dim,
                 actor_lr=1e-3, critic_lr=1e-3,
                 gamma=0.99, tau=0.005, num_quantiles=50):
        self.gamma = gamma
        self.tau = tau
        self.num_quantiles = num_quantiles

        self.actor = Actor(state_dim, action_dim)
        self.actor_target = Actor(state_dim, action_dim)
        self.actor_target.load_state_dict(self.actor.state_dict())

        self.critic = QuantileCritic(state_dim, action_dim, num_quantiles=num_quantiles)
        self.critic_target = QuantileCritic(state_dim, action_dim, num_quantiles=num_quantiles)
        self.critic_target.load_state_dict(self.critic.state_dict())

        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=actor_lr)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_lr)

        self.taus = torch.linspace(0.0, 1.0, num_quantiles + 1)[1:] - 0.5 / num_quantiles

    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0)
        action = self.actor(state)
        return action.detach().cpu().numpy()[0]

    def update(self, replay_buffer, batch_size=64, kappa=1.0):
        state, action, reward, next_state, done = replay_buffer.sample(batch_size)
        transformed_reward = cpt_transform_tensor(reward)
        current_quantiles = self.critic(state, action)
        
        with torch.no_grad():
            next_action = self.actor_target(next_state)
            next_quantiles = self.critic_target(next_state, next_action)
            target_quantiles = transformed_reward.unsqueeze(1) + (1 - done) * self.gamma * next_quantiles
        
        critic_loss = quantile_regression_loss(current_quantiles, target_quantiles, self.taus, kappa=kappa)
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        
        actor_loss = -self.critic(state, self.actor(state)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
        
        return critic_loss.item(), actor_loss.item()

class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def add(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        indices = torch.randint(0, len(self.buffer), (batch_size,))
        batch = [self.buffer[i] for i in indices]
        state, action, reward, next_state, done = zip(*batch)
        state = torch.stack(state)
        action = torch.stack(action)
        reward = torch.stack(reward).squeeze()
        next_state = torch.stack(next_state)
        done = torch.stack(done).squeeze()
        return state, action, reward, next_state, done
    
    def size(self):
        return len(self.buffer)


: 

In [ ]:
## --- PettingZoo Competitive Environment Integration ---

env = simple_tag_v2.env()
env.reset()

# Create a dictionary to hold an agent and replay buffer for each PettingZoo agent
agents = {}
replay_buffers = {}
for agent in env.agents:
    state_dim = env.observation_space(agent).shape[0]
    action_dim = env.action_space(agent).shape[0]
    agents[agent] = CPTQuantileDDPG(state_dim, action_dim, num_quantiles=50)
    replay_buffers[agent] = ReplayBuffer(capacity=100000)

min_buffer_size = 64

num_episodes = 50
for episode in range(num_episodes):
    env.reset()
    # Get initial observations for all agents
    observations = {agent: env.observe(agent) for agent in env.agents}
    
    # Use PettingZoo's agent-iterative API
    for agent in env.agent_iter():
        obs, reward, done_flag, info = env.last()
        if done_flag:
            action = None
        else:
            obs_np = np.array(obs, dtype=np.float32)
            action = agents[agent].select_action(obs_np)
            action = np.clip(action, env.action_space(agent).low, env.action_space(agent).high)
        env.step(action)
        
        # Get the new observation for the current agent
        new_obs = env.observe(agent)
        
        # Convert to tensors
        state_tensor = torch.tensor(np.array(obs, dtype=np.float32))
        action_tensor = torch.tensor(action, dtype=torch.float32) if action is not None else torch.zeros(env.action_space(agent).shape, dtype=torch.float32)
        reward_tensor = torch.tensor([reward], dtype=torch.float32)
        next_state_tensor = torch.tensor(np.array(new_obs, dtype=np.float32))
        done_tensor = torch.tensor([float(done_flag)], dtype=torch.float32)
        
        # Add the transition to the replay buffer for this agent
        replay_buffers[agent].add(state_tensor, action_tensor, reward_tensor, next_state_tensor, done_tensor)
        
        # Update if sufficient samples are available
        if replay_buffers[agent].size() >= min_buffer_size:
            critic_loss, actor_loss = agents[agent].update(replay_buffers[agent], batch_size=64)
            print(f"Episode {episode} | Agent {agent} | Critic Loss: {critic_loss:.4f} | Actor Loss: {actor_loss:.4f}")
